# sql.py
- 1. 모듈 구성
    + Save_DB : 데이터베이스 생성 및 데이터베이스에 데이터 저장 자동화 코드
    + Load_DB : 데이터베이스에서 데이터 조회 및 판다스 데이터테이블 추출 자동화 코드

- 2. 모듈 사용방법
    + 1. sql.Save_DB(data, weather)
        * data는 사내급식 관련 csv
        * weather는 기상청 날씨 csv
        * db 파일을 안만들었으면 sql.Save_DB(data, weather).make_db() 명령어 입력
        * 데이터 가공 후 데이터베이스에 데이터값들 저장
            + sql.Save_DB(data, weather).save() : sql에 저장하기 위해 데이터 전처리 및 csv로 저장해주는 자동화 코드
        * csv 파일 읽어서 db에 데이터 입력
            + sql.Save_DB(data, weather).input()
    + 2. sql.Load_DB() : DB에 있는 데이터테이블을 판다스 데이터프레임으로 추출하는 자동화 코드

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from os import mkdir
import csv

In [2]:
import sql

# 0. 모듈 호출

In [3]:
data = pd.read_csv("../data/train.csv")
weather = pd.read_csv("../data/weather.csv", encoding='euc-kr')
Save_DB = sql.Save_DB(data, weather)

# 1. DB 생성
- OK

In [4]:
# sqlite3로 db 파일 생성 / db 내 테이블 만들기 함수
# 폴더 내 data/team3.db가 생성
Save_DB.make_db()

# 2. 데이터 전처리
- OK

In [5]:
prepro = Save_DB.make_csv()

/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


In [6]:
prepro

,datetime,weekdays,worker_number,real_number,vacation_number,biztrip_number,overtime_number,telecom_number,lunch_number,dinner_number,...,lunch_main,dinner_rice,dinner_soup,dinner_main,temperature,rain,wind,humidity,discomfort_index,perceived_temperature
0,2016-02-01,monday,2601,2401.0,50,150,238,0.0,1039.0,331.0,...,쇠불고기,밥,육개장,자반고등어구이,-0.6,0.0,1.3,43.9,39.275534,0.641548
1,2016-02-02,tuesday,2601,2378.0,50,173,319,0.0,867.0,560.0,...,가자미튀김,콩나물밥*양념장,어묵국,유산슬,-2.3,0.0,0.8,47.5,36.562925,-0.160634
2,2016-02-03,wednesday,2601,2365.0,56,180,111,0.0,1017.0,573.0,...,치킨핑거,밥,청국장찌개,황태양념구이,-1.7,0.0,0.6,57.1,35.796707,0.964631
3,2016-02-04,thursday,2601,2277.0,104,220,355,0.0,978.0,525.0,...,주꾸미볶음,미니김밥*겨자장,우동,멕시칸샐러드,-0.2,0.0,0.7,53.4,38.396068,2.181494
4,2016-02-05,friday,2601,2142.0,278,181,34,0.0,925.0,330.0,...,돈육씨앗강정,밥,차돌박이찌개,닭갈비,1.3,0.0,1.0,44.4,41.575228,3.073400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,wednesday,2983,2319.0,75,198,4,391.0,1093.0,421.0,...,수제함박스테이크,김치볶음밥,미니쫄우동*맛살튀김,브로콜리깨소스무침,-0.4,0.0,0.5,64.0,36.570560,2.553014
1201,2021-01-21,thursday,2983,2309.0,92,231,462,351.0,832.0,353.0,...,동파육,밥,쇠고기무국,삼치양념구이,5.2,1.7,0.6,77.8,43.391744,7.774119
1202,2021-01-22,friday,2983,2177.0,255,248,1,303.0,579.0,217.0,...,요거닭,밥,수제비국,수제맛쵸킹탕수육,7.0,1.2,0.5,97.5,44.784250,9.778207
1203,2021-01-25,monday,2983,2396.0,107,153,616,327.0,1145.0,502.0,...,돈육간장불고기,밥,열무된장국,장어강정*데리야끼소스,6.3,0.0,1.0,75.8,45.291246,8.163400


In [7]:
hr, lunch, dinner, weather, calender = Save_DB.split()

/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'
/Users/hoon/Desktop/Data/vscode/team_project/Forecast_final/module/sql.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dinner_rice'][index] = '밥'


KeyError: '일시'

In [8]:
weather

,기온,강수량,풍속,습도,일자,불쾌지수,체감온도
0,-0.6,0.0,1.3,43.9,2016-02-01,39.275534,0.641548
1,-2.3,0.0,0.8,47.5,2016-02-02,36.562925,-0.160634
2,-1.7,0.0,0.6,57.1,2016-02-03,35.796707,0.964631
3,-0.2,0.0,0.7,53.4,2016-02-04,38.396068,2.181494
4,1.3,0.0,1.0,44.4,2016-02-05,41.575228,3.073400
...,...,...,...,...,...,...,...
2171,0.1,0.0,2.3,44.4,2022-01-11,40.075756,0.236622
2172,-2.2,0.0,1.1,42.0,2022-01-12,37.597240,-0.677721
2173,-2.4,0.0,2.0,43.8,2022-01-13,37.051912,-2.138365
2174,-3.0,0.0,0.5,47.5,2022-01-14,35.666750,0.014433


In [ ]:
hr.info()

In [ ]:
lunch.info()

In [ ]:
dinner.info()

In [ ]:
weather.info()

In [ ]:
calender.info()

# 3. DB 저장
- 1. data 폴더에 csv 파일 생성 확인
- 2. 가능하면 sqlite로 db에 잘 저장되었는지 확인
- 3. 중복 입력을 막기 위해  datetime 필드는 key-value로 설정해주자

### 1. save
- sql.Save_DB(data, weather).save()
- data 폴더에 데이터 csv파일 잘 저장되는지 확인

In [17]:
Save_DB.save()

### 2. input
- sql.Save_DB(data, weather).input()
- 이 함수들은 경로 지정이 중요
- 에러 발생시 sql.py에서 저장경로와 csv 파일을 읽는 경로를 확인

In [18]:
Save_DB.input()

# 4. Load_DB
- DB에서 데이터 추출해 판다스 데이터프레임으로 파이썬 위에 올려두기

In [ ]:
df = sql.Load_DB()
df

# 5. sql.py 모듈 진행 코드
- 식수인원 프로젝트 sql 관련 모듈은 다음과 같이 진행하면 됩니다.

In [ ]:
data = pd.read_csv("../data/train.csv")
weather = pd.read_csv("../data/weather.csv", encoding='euc-kr')
Save_DB = sql.Save_DB(data, weather)

### 1. DB 만들기. DB 있으면 생략
Save_DB.make_db()

### 2. SQL에 맞게 데이터프레임 조작
Save_DB.save()
Save_DB.input()

### 3. SQLite3로 DB에서 데이터프레임 추출
df = sql.Load_DB()
df

